In [8]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
    'qty':'{:,}','price':'{:.2f}','target':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd'.split()
cash = 720_000
cash

720000

### Print to verify before upload file

In [23]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [24]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

orders[cols].to_csv(output_file, header=False, index=False)
orders[cols].to_csv(data_file, header=False, index=False)
orders[cols].to_csv(box_file, header=False, index=False)

### End of transactional process

In [25]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,DOHOME,"4,000",22.90,"91,600.00",RD05%,SET100,2
1,B,NOBLE,"21,000",6.00,"126,000.00",RD10%,SET,2
2,B,ROJNA,"18,000",6.30,"113,400.00",6L,SET,2
3,B,SIS,"3,000",42.00,"126,000.00",DFM10%,SET,2
4,B,TISCO,"2,000",94.50,"189,000.00",RD03%,SET100,2
5,S,DCC,"40,000",2.90,"116,000.00",COST,SET,2
6,S,DIF,"10,000",14.20,"142,000.00",CUT,SET,2
7,S,IVL,"3,000",44.00,"132,000.00",5%,SET50,2
8,S,KBANK,800,143.50,"114,800.00",5%,SET50,2
9,S,VNT,"4,500",38.75,"174,375.00",15%,SET,2


In [26]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
B    646000.0
S    679175.0
Name: amount, dtype: float64

In [27]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,B,DOHOME,"4,000",22.90,"91,600.00",RD05%,SET100,2
1,B,NOBLE,"21,000",6.00,"126,000.00",RD10%,SET,2
2,B,ROJNA,"18,000",6.30,"113,400.00",6L,SET,2
3,B,SIS,"3,000",42.00,"126,000.00",DFM10%,SET,2
4,B,TISCO,"2,000",94.50,"189,000.00",RD03%,SET100,2


In [28]:
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(720000, 646000.0, 74000.0)

In [29]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
5,S,DCC,"40,000",2.90,"116,000.00",COST,SET,2
6,S,DIF,"10,000",14.20,"142,000.00",CUT,SET,2
7,S,IVL,"3,000",44.00,"132,000.00",5%,SET50,2
8,S,KBANK,800,143.50,"114,800.00",5%,SET50,2
9,S,VNT,"4,500",38.75,"174,375.00",15%,SET,2


In [30]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(720000, 679175.0, 1399175.0)

### After call ord-log

In [31]:
def categorise(row):  
    if row['col0'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [37]:
df = pd.read_csv('../data/orders-log.csv', header=None, prefix = 'col',usecols=[0,1,2,3,4,5,6,7,8,9,11])
df.rename(columns = {'col1': 'name', 'col2': 'spd','col3': 'reason', 'col4': 'market','col5': 'qty', 'col6': 'target','col7': 'current', 'col8': 'chg', 'col9': 'percent', 'col11': 'active'}, inplace = True)
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [38]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [39]:
mb = (df.trans == 'Buy') & (df.spd >= -5)
df[mb].sort_values(['name'],ascending=[True])

,col0,name,spd,reason,market,qty,target,current,chg,percent,active,trans,amount
0,B,DOHOME,-5,RD05%,SET100,4000,22.9,23.4,0.20,+0.86%,2,Buy,91600.0
2,B,NOBLE,-4,RD10%,SET,21000,6.0,6.2,0.05,+0.81%,2,Buy,126000.0


In [36]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [40]:
ms = (df.trans == 'Sell') & (df.spd <= 5)
df[ms].sort_values(['name'],ascending=[True])

,col0,name,spd,reason,market,qty,target,current,chg,percent,active,trans,amount
10,S,BGRIM,4,5%,SET50,3000,42.00,41.00,1.00,+2.50%,1,Sell,126000.0
12,S,DCC,3,COST,SET,40000,2.90,2.84,0.00,0.00%,2,Sell,116000.0
13,S,DIF,2,CUT,SET,10000,14.20,14.00,0.00,0.00%,2,Sell,142000.0
14,S,IVL,3,5%,SET50,3000,44.00,43.25,0.25,+0.58%,2,Sell,132000.0
15,S,KBANK,4,5%,SET50,800,143.50,141.50,-0.50,-0.35%,2,Sell,114800.0
23,S,VNT,1,15%,SET,4500,38.75,38.50,0.00,0.00%,2,Sell,174375.0


In [41]:
df[mb | ms].sort_values(['col0','name'],ascending=[True,True])

,col0,name,spd,reason,market,qty,target,current,chg,percent,active,trans,amount
0,B,DOHOME,-5,RD05%,SET100,4000,22.90,23.40,0.20,+0.86%,2,Buy,91600.0
2,B,NOBLE,-4,RD10%,SET,21000,6.00,6.20,0.05,+0.81%,2,Buy,126000.0
10,S,BGRIM,4,5%,SET50,3000,42.00,41.00,1.00,+2.50%,1,Sell,126000.0
12,S,DCC,3,COST,SET,40000,2.90,2.84,0.00,0.00%,2,Sell,116000.0
13,S,DIF,2,CUT,SET,10000,14.20,14.00,0.00,0.00%,2,Sell,142000.0
14,S,IVL,3,5%,SET50,3000,44.00,43.25,0.25,+0.58%,2,Sell,132000.0
15,S,KBANK,4,5%,SET50,800,143.50,141.50,-0.50,-0.35%,2,Sell,114800.0
23,S,VNT,1,15%,SET,4500,38.75,38.50,0.00,0.00%,2,Sell,174375.0


In [42]:
df[mb | ms].shape[0]

8

In [43]:
mask = (df.col0 == 'B')
df[mb | ms][mask].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

C:\Users\User\AppData\Local\Temp/ipykernel_25716/1898287905.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[mb | ms][mask].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)


,name,trans,amount
2,NOBLE,Buy,"126,000.00"
0,DOHOME,Buy,"91,600.00"


In [44]:
df_out = df[mb | ms][colt]
#df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd
0,Buy,DOHOME,"4,000",22.90,2,-5
2,Buy,NOBLE,"21,000",6.00,2,-4
10,Sell,BGRIM,"3,000",42.00,1,4
12,Sell,DCC,"40,000",2.90,2,3
13,Sell,DIF,"10,000",14.20,2,2
14,Sell,IVL,"3,000",44.00,2,3
15,Sell,KBANK,800,143.50,2,4
23,Sell,VNT,"4,500",38.75,2,1


In [45]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=False, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=False, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=False, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

### End of Evening Process